Test on piecewise constant curves in so3

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats('pdf', 'svg')

# torch.set_default_dtype(torch.float64)
from deepthermal.FFNN_model import fit_FFNN, FFNN, init_xavier
from deepthermal.validation import create_subdictionary_iterator, k_fold_cv_grid, add_dictionary_iterators
from deepthermal.plotting import plot_result, plot_model_1d

from deep_reparametrization.reparametrization import (
    get_elastic_metric_loss,
    compute_loss_reparam,
)
from deep_reparametrization.helpers import get_pc_curve_from_data, get_pl_curve_from_data
from deep_reparametrization.ResNET import ResNET



In [3]:
import sys

sys.path.append("../../")
from animation.animation_manager import fetch_animations, unpack
from so3.curves import move_origin_to_zero, dynamic_distance
from so3.helpers import crop_curve
from so3.dynamic_distance import find_optimal_diffeomorphism, create_shared_parameterization
from so3.clustering.id_set import crop_curve_based_on_id, get_id_set
from so3.transformations import skew_to_vector, SRVT
from so3 import animation_to_SO3

max_frame_count = 180

id_set = get_id_set()
print("Load data")

data = [fetch_animations(1, file_name="39_02.amc"),  #walk 6.5 steps
        fetch_animations(1, file_name="35_26.amc"),  # run/jog 3 steps
        fetch_animations(1, file_name="16_35.amc")  # run/jog 3 steps
        ]

# walk
subject, animation, desc0 = unpack(data[2])
curve_full = animation_to_SO3(subject, animation)
curve = crop_curve(curve_full, stop=240)  #first 2 seconds
c_0 = move_origin_to_zero(curve)
print(desc0)

# run
subject, animation, desc1 = unpack(data[1])
curve_full = animation_to_SO3(subject, animation)
curve = crop_curve(curve_full, stop=240)  #first 2 seconds
c_1 = move_origin_to_zero(curve)
print(desc1)
print(c_0.shape)
print(c_1.shape)

Load data
run/jog
run/jog
(23, 161, 3, 3)
(23, 137, 3, 3)


In [4]:
#calculate distances
I0 = np.linspace(0, 1, c_0.shape[1])
I1 = np.linspace(0, 1, c_1.shape[1])
q_data_ = skew_to_vector(SRVT(c_0, I0))
r_data_ = skew_to_vector(SRVT(c_1, I1))
I, q_data, r_data = create_shared_parameterization(q0=q_data_, q1=r_data_, I0=I0, I1=I1)
shared_frames = I.shape[0]
q_func = get_pc_curve_from_data(data=q_data)
r_func = get_pc_curve_from_data(data=r_data)


In [5]:
I1_new = find_optimal_diffeomorphism(q0=q_data, q1=r_data, I0=I, I1=I, depth=10)


In [6]:
########
DIR = "../figures/curve_so3/"
SET_NAME = "pc_eks_2"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########

FOLDS = 1
N = shared_frames  # training points internal

loss_func = get_elastic_metric_loss(r=r_func, constrain_cost=1e3, verbose=False)
penalty_free_loss_func = get_elastic_metric_loss(r=r_func, constrain_cost=0, verbose=False)
lr_scheduler = lambda optimizer: torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7, patience=200,
                                                                            verbose=True)
MODEL_PARAMS = {
    "model": [FFNN, ResNET],
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": ["tanh", "relu"],
    "n_hidden_layers": [2],  #,8,16,64],
}
# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "neurons": [16]
}
TRAINING_PARAMS = {
    "batch_size": [N],
    "regularization_param": [0],
    "compute_loss": [compute_loss_reparam],
    "loss_func": [loss_func],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["ADAM"],
    "num_epochs": [2000, ],
    "learning_rate": [0.01],
    "lr_scheduler": [lr_scheduler]
}


In [7]:
x_train = torch.linspace(0, 1, N, requires_grad=True).unsqueeze(1)
q_train = torch.tensor(q_data)
data = TensorDataset(x_train, q_train)

model_params_iter = create_subdictionary_iterator(MODEL_PARAMS)
model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(TRAINING_PARAMS_EXPERIMENT, product=False)
exp_training_params_iter = add_dictionary_iterators(training_exp_iter, training_params_iter)

Do the actual training

In [8]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_FFNN,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=5
)


Running model (trial=0, mod=0, k=0):
Parameters: ({'model': <class 'deepthermal.FFNN_model.FFNN'>, 'input_dimension': 1, 'output_dimension': 1, 'activation': 'tanh', 'n_hidden_layers': 2, 'neurons': 16}, {'batch_size': 289, 'regularization_param': 0, 'compute_loss': <function compute_loss_reparam at 0x123cffe50>, 'loss_func': <function get_elastic_metric_loss.<locals>.elastic_metric_loss at 0x123eef430>, 'optimizer': 'ADAM', 'num_epochs': 2000, 'learning_rate': 0.01, 'lr_scheduler': <function <lambda> at 0x123eef5e0>})
################################  0  ################################
Training Loss:  46582.25
################################  100  ################################
Training Loss:  123.65589905
################################  200  ################################
Training Loss:  121.68216705
Epoch   250: reducing learning rate of group 0 to 7.0000e-03.
################################  300  ################################
Training Loss:  120.83898926
##############

In [10]:
# plotting
disc_points = x_train.detach()

plot_kwargs = {
    "x_test": disc_points,
    "x_train": disc_points,
    "y_train": I1_new,
    "x_axis": "t",
    "y_axis": "$\\varphi(t)$",
    "compare_label": "DP solution"
}
plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
    plot_function=plot_model_1d,
    function_kwargs=plot_kwargs,
)